In [19]:
import zipfile
import numpy as np
import tensorflow as tf

path_to_glove = "/home/younseun/columbus_study/04_learning_tensorflow/06__word_embeddings_and_rnns/glove_data/glove.840B.300d.zip"
PRE_TRAINED = True
GLOVE_SIZE = 300
batch_size = 128
embedding_dimension = 64
num_classes = 2
hidden_layer_size = 32
times_steps = 6

In [20]:
Sample_cnt = 10000
odd_sentences=[]
even_sentences=[]
seqlens=[]

digit_to_word_map = {1:"One",2:"Two",3:"Three",4:"Four",5:"Five",6:"Six",7:"Seven",8:"Eight",9:"Nine",}
digit_to_word_map[0]="PAD"
for i in range(Sample_cnt):
    rand_seq_len = np.random.choice(range(3,7))
    seqlens.append(rand_seq_len)
    rand_odd_ints = np.random.choice(range(1,10,2), rand_seq_len)
    rand_even_ints = np.random.choice(range(2,10,2), rand_seq_len)

    # 패딩
    if rand_seq_len < 6:
        rand_odd_ints = np.append(rand_odd_ints, [0]*(6-rand_seq_len))
        rand_even_ints = np.append(rand_even_ints, [0]*(6-rand_seq_len))
    odd_sentences.append(" ".join([digit_to_word_map[r] for r in rand_odd_ints]))
    even_sentences.append(" ".join([digit_to_word_map[r] for r in rand_even_ints]))
data = odd_sentences + even_sentences
len(odd_sentences),len(even_sentences),len(data)

# 홀수, 짝수 시퀀스의 seq 길이(2배 증폭)
seqlens*=2

# 단어를 인덱스에 매핑
word2index_map = {}
index = 0
for sent in data:
    for word in sent.lower().split():
        if word not in word2index_map:
            word2index_map[word] = index
            index += 1

# 역방향 매핑
index2word_map = {index: word for word, index in word2index_map.items()}
vocabulary_size = len(index2word_map)
print('word2index_map :{}'.format(word2index_map))
print('index2word_map :{}'.format(index2word_map))
print('vocabulary_size :{}'.format(vocabulary_size))

# one hot encoding : 짝수=[1,0], 홀수=[0,1]
labels = [1]*Sample_cnt + [0]*Sample_cnt
for i in range(len(labels)):
    label = labels[i]
    one_hot_encoding = [0]*2
    one_hot_encoding[label] = 1
    labels[i] = one_hot_encoding
print('앞쪽')
print('   >>> data : {}'.format(data[:3]))
print('   >>> one-hot-encoding : {}'.format(labels[:3]))
print('뒤쪽')
print('   >>> data : {}'.format(data[Sample_cnt*2-4:-1]))
print('   >>> one-hot-encoding : {}'.format(labels[Sample_cnt*2-4:-1]))

word2index_map :{'nine': 0, 'three': 1, 'five': 2, 'one': 3, 'seven': 4, 'pad': 5, 'two': 6, 'eight': 7, 'four': 8, 'six': 9}
index2word_map :{0: 'nine', 1: 'three', 2: 'five', 3: 'one', 4: 'seven', 5: 'pad', 6: 'two', 7: 'eight', 8: 'four', 9: 'six'}
vocabulary_size :10
앞쪽
   >>> data : ['Nine Nine Three Three Five One', 'Seven Five One One Five Nine', 'Nine Seven Five One Seven PAD']
   >>> one-hot-encoding : [[0, 1], [0, 1], [0, 1]]
뒤쪽
   >>> data : ['Four Two Eight Eight Six Eight', 'Six Two Eight PAD PAD PAD', 'Two Four Six Two Eight PAD']
   >>> one-hot-encoding : [[1, 0], [1, 0], [1, 0]]


In [21]:
word2index_map = {}
index = 0
for sent in data:
    for word in sent.split():
        if word not in word2index_map:
            word2index_map[word] = index
            index += 1
index2word_map = {index:word for word, index in word2index_map.items()}            
vocabulary_size = len(index2word_map)        
print('{}'.format(vocabulary_size))
print('word2index_map : {}'.format(word2index_map))
print('index2word_map : {}'.format(index2word_map))
    

10
word2index_map : {'Nine': 0, 'Three': 1, 'Five': 2, 'One': 3, 'Seven': 4, 'PAD': 5, 'Two': 6, 'Eight': 7, 'Four': 8, 'Six': 9}
index2word_map : {0: 'Nine', 1: 'Three', 2: 'Five', 3: 'One', 4: 'Seven', 5: 'PAD', 6: 'Two', 7: 'Eight', 8: 'Four', 9: 'Six'}


In [44]:
def get_glove(path_to_glove, word2index_map):
    first = 0
    embedding_weights = {}
    count_all_words = 0
    with zipfile.ZipFile(path_to_glove) as z:
        with z.open("glove.840B.300d.txt") as f:
            for line in f:
                vals = line.split()
                word = str(vals[0].decode("utf-8"))
                first += 1
                if word in word2index_map:
                    print('{} {} 번째'.format(word, first))
                    count_all_words += 1
                    coefs = np.asarray(vals[1:], dtype='float32')
                    coefs /= np.linalg.norm(coefs)
                    embedding_weights[word] = coefs
                    if first == 1:
                        print(coefs)
                
                if count_all_words == len(word2index_map) - 1:
                    break
                        
                #if word == 'PAD' or first % 10000 == 0:
                #    print('>>> {} {} 번째'.format(word, first))
                #if first >= 1000000:
                #    break
    return embedding_weights



In [45]:
word2embedding_dict = get_glove(path_to_glove, word2index_map)
print('word2embedding_dict shape : {}'.format(word2embedding_dict['Eight'].shape))

One 290 번째
Two 1054 번째
Three 2026 번째
Four 2954 번째
Five 3663 번째
Six 4937 번째
Seven 6228 번째
Nine 9432 번째
Eight 10606 번째
word2embedding_dict shape : (300,)


In [27]:
embedding_matrix = np.zeros((vocabulary_size, GLOVE_SIZE))
print('{} {}'.format(vocabulary_size, GLOVE_SIZE))
embedding_matrix
type(word2index_map)

10 300


dict

In [28]:
for word, index in word2index_map.items():
    if not word == "PAD":
        word_embedding = word2embedding_dict[word]
        embedding_matrix[index,:] = word_embedding
print('embedding_matrix type:{} data:{}'.format(type(embedding_matrix),embedding_matrix))




embedding_matrix type:<class 'numpy.ndarray'> data:[[ 0.00501343 -0.02988535 -0.0271745  ...  0.01667927  0.00738378
  -0.10879323]
 [-0.03824107 -0.02461622 -0.07610063 ... -0.05147701  0.04443772
  -0.075308  ]
 [-0.02271027  0.00910496 -0.03779484 ... -0.04140582  0.04717579
  -0.06907098]
 ...
 [ 0.00242988  0.0330428  -0.01491071 ... -0.07107946  0.02411918
  -0.09388998]
 [-0.00793781  0.0166579  -0.03776361 ... -0.04945398  0.0618212
  -0.05904504]
 [-0.0296887   0.0435931  -0.05909128 ... -0.01779287  0.02460201
  -0.07145758]]


In [46]:
# Train Set, Test Set 만들기
Sample_cnt = 10000
data_indices = list(range(len(data)))
np.random.shuffle(data_indices)
data_shuffle = np.array(data)[data_indices]
labels_shuffle = np.array(labels)[data_indices]
seqlens_shuffle = np.array(seqlens)[data_indices]
#print('data_shuffle : {}'.format(data_shuffle[:5]))
#print('labels_shuffle : {}'.format(labels_shuffle[:5]))
#print('seqlens_shuffle : {}'.format(seqlens_shuffle[:5]))

train_x = data_shuffle[:Sample_cnt]
train_y = labels_shuffle[:Sample_cnt]
train_seqlens = seqlens_shuffle[:Sample_cnt]

test_x = data_shuffle[Sample_cnt:]
test_y = labels_shuffle[Sample_cnt:]
test_seqlens = seqlens_shuffle[Sample_cnt:]


In [47]:
# bath_size만큼 데이터를 Random하게 제공
def get_sentence_batch(batch_size, data_x,
                       data_y, data_seqlens):
    instance_indices = list(range(len(data_x)))
    np.random.shuffle(instance_indices)
    batch = instance_indices[:batch_size]
    x = [[word2index_map[word] for word in data_x[i].lower().split()]
         for i in batch]
    y = [data_y[i] for i in batch]
    seqlens = [data_seqlens[i] for i in batch]
    return x, y, seqlens

In [48]:
# train data의 placeholder
_inputs = tf.placeholder(tf.int32, shape=[batch_size, times_steps])
embedding_placeholder = tf.placeholder(tf.float32, shape=[vocabulary_size, GLOVE_SIZE])
_labels = tf.placeholder(tf.int32, shape=[batch_size, num_classes])
_seqlens = tf.placeholder(tf.int32, shape=[batch_size])

In [57]:
if PRE_TRAINED:
        embeddings = tf.Variable(tf.constant(0.0, shape=[vocabulary_size, GLOVE_SIZE]),
                                 trainable=True)
        # if using pre-trained embeddings, assign them to the embeddings variable
        embedding_init = embeddings.assign(embedding_placeholder)
        embed = tf.nn.embedding_lookup(embeddings, _inputs)
        print('embedding_placeholder:{}\nembedding_init:{}\nembed:{}'.format(embedding_placeholder,embedding_init,embed))
else:
        embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size,
                               embedding_dimension],
                              -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, _inputs)

embedding_placeholder:Tensor("Placeholder_1:0", shape=(10, 300), dtype=float32)
embedding_init:Tensor("Assign_8:0", shape=(10, 300), dtype=float32_ref)
embed:Tensor("embedding_lookup_8:0", shape=(128, 6, 300), dtype=float32)


In [66]:
with tf.name_scope("biGRU"):
    with tf.variable_scope('forward'):
        gru_fw_cell = tf.contrib.rnn.GRUCell(hidden_layer_size)
        gru_fw_cell = tf.contrib.rnn.DropoutWrapper(gru_fw_cell)

    with tf.variable_scope('backward'):
        gru_bw_cell = tf.contrib.rnn.GRUCell(hidden_layer_size)
        gru_bw_cell = tf.contrib.rnn.DropoutWrapper(gru_bw_cell)

    with tf.variable_scope("biGRU", reuse=tf.AUTO_REUSE):
        outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw=gru_fw_cell,
                                                          cell_bw=gru_bw_cell,
                                                          inputs=embed,
                                                          sequence_length=_seqlens,
                                                          dtype=tf.float32,
                                                          scope="biGRU")
        states = tf.concat(values=states, axis=1)

In [71]:
weights = {'linear_layer': tf.Variable(tf.truncated_normal([2*hidden_layer_size,
                                                    num_classes],
                                                    mean=0, stddev=.01))}
biases = {'linear_layer': tf.Variable(tf.truncated_normal([num_classes],
                                                    mean=0, stddev=.01))}
print('{}\n{}'.format(weights, biases))

{'linear_layer': <tf.Variable 'Variable_16:0' shape=(64, 2) dtype=float32_ref>}
{'linear_layer': <tf.Variable 'Variable_17:0' shape=(2,) dtype=float32_ref>}


In [73]:
# extract the final state and use in a linear layer
final_output = tf.matmul(states,
                         weights["linear_layer"]) + biases["linear_layer"]

softmax = tf.nn.softmax_cross_entropy_with_logits(logits=final_output,
                                                  labels=_labels)
cross_entropy = tf.reduce_mean(softmax)

train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(_labels, 1),
                              tf.argmax(final_output, 1))
accuracy = (tf.reduce_mean(tf.cast(correct_prediction,
                                   tf.float32)))*100

In [75]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(embedding_init,
             feed_dict={embedding_placeholder: embedding_matrix})
    for step in range(1000):
        x_batch, y_batch, seqlen_batch = get_sentence_batch(batch_size,
                                                            train_x, train_y,
                                                            train_seqlens)
        sess.run(train_step, feed_dict={_inputs: x_batch, _labels: y_batch,
                                        _seqlens: seqlen_batch})

        if step % 100 == 0:
            acc = sess.run(accuracy, feed_dict={_inputs: x_batch,
                                                _labels: y_batch,
                                                _seqlens: seqlen_batch})
            print("Accuracy at %d: %.5f" % (step, acc))

    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings),
                                 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    normalized_embeddings_matrix = sess.run(normalized_embeddings)

    for test_batch in range(5):
        x_test, y_test, seqlen_test = get_sentence_batch(batch_size,
                                                         test_x, test_y,
                                                         test_seqlens)
        batch_pred, batch_acc = sess.run([tf.argmax(final_output, 1), accuracy],
                                         feed_dict={_inputs: x_test,
                                                    _labels: y_test,
                                                    _seqlens: seqlen_test})
        print("Test batch accuracy %d: %.5f" % (test_batch, batch_acc))

ref_word = normalized_embeddings_matrix[word2index_map["Three"]]

cosine_dists = np.dot(normalized_embeddings_matrix, ref_word)
ff = np.argsort(cosine_dists)[::-1][1:10]
for f in ff:
    print(index2word_map[f])
    print(cosine_dists[f])
 

Accuracy at 0: 47.65625
Accuracy at 100: 100.00000
Accuracy at 200: 100.00000
Accuracy at 300: 100.00000
Accuracy at 400: 100.00000
Accuracy at 500: 100.00000
Accuracy at 600: 100.00000
Accuracy at 700: 100.00000
Accuracy at 800: 100.00000
Accuracy at 900: 100.00000
Test batch accuracy 0: 100.00000
Test batch accuracy 1: 100.00000
Test batch accuracy 2: 100.00000
Test batch accuracy 3: 100.00000
Test batch accuracy 4: 100.00000
Three
1.0
Five
0.9507418
Seven
0.91235805
Nine
0.87779546
One
0.8678094
Four
-0.07850474
Two
-0.08900523
Six
-0.10796718
Eight
-0.11757462
